In [14]:
import json
import re
import pandas as pd
import datetime
import os
import time
import glob

In [15]:
summary = {"File":[], 
           "Start Time":[], 
           "End Time":[],
           "Timestamp Length(ms)":[],
           "Total Timestamps":[],
           "0 source detected":[],
           "1 source detected":[],
           "2 source detected":[],
           "3 source detected":[],
           "4 source detected":[]}

In [16]:
# Extract time information of each recording from the log file
def timeExtract(filename):
    with open(filename, 'rb') as f:
        # Start counting from the last byte
        counter = 1
        # Go to the 2nd byte before the end of the last line
        f.seek(-2, 2) 
        while f.read(1) != b'\n':
            f.seek(-2, 1)
            counter=counter+1
        endTime_line = f.readline().decode()
        # Go to the 2nd byte before the end of the last second line
        f.seek(-counter-2, 2)
        while f.read(1) != b'\n':
            f.seek(-2, 1)
        startTime_line = f.readline().decode()

    return [startTime_line, endTime_line]

# Calculate duration of each recording in microseconds
def durationinMicroseconds(filename):
    startTime = timeExtract(filename)[0].split()[2:]
    endTime = timeExtract(filename)[1].split()[2:]
    startTimeStr = startTime[0] + ' ' + startTime[1]
    endTimeStr = endTime[0] + ' ' + endTime[1]
    T1 = datetime.datetime.strptime(startTimeStr, '%Y-%m-%d %H:%M:%S.%f')
    T2 = datetime.datetime.strptime(endTimeStr, '%Y-%m-%d %H:%M:%S.%f')
    delta = T2-T1
    duration = delta.seconds*1000000 + delta.microseconds
    
    return duration, T1, T2

def appendSummary(summary,filename,startTime,endTime,t,active):
    summary["File"].append(filename.split('/')[-1])
    summary["Start Time"].append(str(startTime))
    summary["End Time"].append(str(endTime))
    summary["Timestamp Length(ms)"].append(round(t/1000,4))
    summary["Total Timestamps"].append(len(active))
    summary["0 source detected"].append(active.count(0))
    summary["1 source detected"].append(active.count(1))
    summary["2 source detected"].append(active.count(2))
    summary["3 source detected"].append(active.count(3))
    summary["4 source detected"].append(active.count(4))

In [18]:
for filename in glob.glob("/Users/hyh/Google Drive/ODAS/recordings3/*.log"):
    with open(filename, 'r') as f:
        firstline = f.readline()
    if firstline == "SST log contains no useful data\n":
        pass
    else:
        with open(filename, 'r') as f:
            text = f.read()
        # Use repex to store blocks of data into a list
        data = re.split('(?<=})\n(?={)', text) 
        # Delete the time info from the last data block
        tmp = data[-1][:(data[-1].rfind("}")+1)]
        data[-1] = tmp

        # Obtain duration, start time, end time
        duration, startTime, endTime = durationinMicroseconds(filename)
        # Calculate how fast each timestamp is printed
        t = duration/len(data)
        # Extract src keys from all blocks and wrap them in a list
        srcList = [json.loads(block)["src"] for block in data]

        # def foo(src):
        #     return [line["id"] for line in src]
        foo = lambda src: [line["id"] for line in src]
        # Extract id values from all src keys and wrap them in a list
        idList = [foo(src) for src in srcList]
        # Count the number of active sources at each timestamp and wrap them in a list
        active = [4-row.count(0) for row in idList]
        
        appendSummary(summary,filename,startTime,endTime,t,active)

In [19]:
summary3 = pd.DataFrame(summary)

In [20]:
summary3.to_csv('/Users/hyh/Dev/Freund Research Project/ODAS/Files/CSV/'+'ODAS3.csv')

In [115]:
# import sys
# import os
# import hashlib


# def chunk_reader(fobj, chunk_size=1024):
#     """Generator that reads a file in chunks of bytes"""
#     while True:
#         chunk = fobj.read(chunk_size)
#         if not chunk:
#             return
#         yield chunk


# def get_hash(filename, first_chunk_only=False, hash=hashlib.sha1):
#     hashobj = hash()
#     file_object = open(filename, 'rb')

#     if first_chunk_only:
#         hashobj.update(file_object.read(1024))
#     else:
#         for chunk in chunk_reader(file_object):
#             hashobj.update(chunk)
#     hashed = hashobj.digest()

#     file_object.close()
#     return hashed


# def check_for_duplicates(paths, hash=hashlib.sha1):
#     hashes_by_size = {}
#     hashes_on_1k = {}
#     hashes_full = {}

#     for path in paths:
#         for dirpath, dirnames, filenames in os.walk(path):
#             for filename in filenames:
#                 full_path = os.path.join(dirpath, filename)
#                 try:
#                     # if the target is a symlink (soft one), this will 
#                     # dereference it - change the value to the actual target file
#                     full_path = os.path.realpath(full_path)
#                     file_size = os.path.getsize(full_path)
#                 except (OSError,):
#                     # not accessible (permissions, etc) - pass on
#                     continue

#                 duplicate = hashes_by_size.get(file_size)

#                 if duplicate:
#                     hashes_by_size[file_size].append(full_path)
#                 else:
#                     hashes_by_size[file_size] = []  # create the list for this file size
#                     hashes_by_size[file_size].append(full_path)

#     # For all files with the same file size, get their hash on the 1st 1024 bytes
#     for __, files in hashes_by_size.items():
#         if len(files) < 2:
#             continue    # this file size is unique, no need to spend cpy cycles on it

#         for filename in files:
#             try:
#                 small_hash = get_hash(filename, first_chunk_only=True)
#             except (OSError,):
#                 # the file access might've changed till the exec point got here 
#                 continue

#             duplicate = hashes_on_1k.get(small_hash)
#             if duplicate:
#                 hashes_on_1k[small_hash].append(filename)
#             else:
#                 hashes_on_1k[small_hash] = []          # create the list for this 1k hash
#                 hashes_on_1k[small_hash].append(filename)

#     # For all files with the hash on the 1st 1024 bytes, get their hash on the full file - collisions will be duplicates
#     for __, files in hashes_on_1k.items():
#         if len(files) < 2:
#             continue    # this hash of fist 1k file bytes is unique, no need to spend cpy cycles on it

#         for filename in files:
#             try: 
#                 full_hash = get_hash(filename, first_chunk_only=False)
#             except (OSError,):
#                 # the file access might've changed till the exec point got here 
#                 continue

#             duplicate = hashes_full.get(full_hash)
#             if duplicate:
#                 print ("Duplicate found: %s and %s" % (filename, duplicate))
#             else:
#                 hashes_full[full_hash] = filename